# Computing Markov-Perfect Nash Equilibria: Numerical Implications of a Dynamic Differentiated Product Model
## Authors: Ariel Pakes and Paul McGuire

In this pratice, I replicate Pakes and McGuire (1994). I follow Section 3 "Computational Algorithm" of the paper that provides instruction to write down the code to estimate the model developed.

# Packages
You may need to run this part (only once).

In [1]:
using Pkg
# Pkg.add("JuMP")
# Pkg.add("NLsolve")
# Pkg.add("DataFrames")
# Pkg.add("GLPK")
# Pkg.add("Distributions")
# Pkg.add("Plots")
# Pkg.add("Polynomials")
Pkg.status()

    Status `C:\Users\amirh\.julia\environments\v1.3\Project.toml`
  [a93c6f00] DataFrames v0.20.2
  [31c24e10] Distributions v0.23.2
  [60bf3e95] GLPK v0.12.1
  [7073ff75] IJulia v1.20.2
  [4076af6c] JuMP v0.20.1
  [d41bc354] NLSolversBase v7.6.1
  [2774e3e8] NLsolve v4.3.0
  [91a5bcdd] Plots v1.0.11
  [f27b6e38] Polynomials v0.7.0


## Using Packages

In [2]:
using LinearAlgebra
using JuMP
using NLsolve
using DataFrames
using Distributions
using Plots
using Polynomials

We begin with the problem of computing the optimal policies and value function i.e. $\{\mathcal{X}(\omega,s), x(\omega,s), V^e(s,m)\}$. To obtain the equilibrium policies we compute the value of being active for different values of a firm's own, and its competitors' state variables: $V(.,.):\Omega\times S\rightarrow\mathbb{R}$. It is easiest to think of our algorithm as starting with the value function for an evironment that limits the number of active firms to N. We then push $N$ up until it no longer constrains the problem.

Starting with $N=1$, or the monopolist's problem, we look for the value function and the optimal policy when only one firm is allowed to be active. When
$$N=1, S =\{ e(\omega):\omega\in\Omega\}$$

Temporarily assume that the boundaries for $\Omega = [\underline\omega,\underline\omega+1,\dots,\overline\omega]$ are known, and let $k=\overline\omega-\underline\omega$. The Bellman equation for the monopoly case is
$$ V(\omega) = max \{ \Phi, sup_{x\geq0}\pi(\omega)-cx+\beta\sum_\tau V(\omega+\tau)p(\tau|x)\}.$$

The operation $T:\mathbb{R}^k\rightarrow\mathbb{R}^k$ is contraction mapping with modulus $\beta$.
$$ T\mathcal{l}(\omega)=max\{\Phi, sup_{x\geq0}\pi(\omega)-cx+\beta\sum_\tau \mathcal{l}(\omega+\tau)p(\tau|x)\}.$$

As a result, there is unique vector $\{V(\omega)\}\in\mathbb{R}^k$ that satisfies the equation above and it can be calculated as follows. Start with $\mathcal{l}^0(\omega)$, any bounded function from $\Omega$ to $\mathbb{R}$.



## Parameters & Variables

In [3]:
max_iter = 10 # maximum number of iteration allowed
N_x = 200 # number of different values that x (investment) can get
ω_min = 1  # lower bound of productivity
ω_max = 21 # upper bound of productivity
Ω = ω_min:ω_max
k = size(Ω,1)
ω_star = 4
s_e = 4
# stochastic shocks
ν2 = [0;1]
ν1 = [0;1]
ν =  [0;1]
e = [0;1]
τ = ones(4,2)

x = zeros(N_x) #investment
δ = 0.7
β = 0.925
x_e = 0.2
x_el = 0.1
x_eh = 0.3

Φ = 0.1  # scrap value of the firm
M = 5 # Market size
a = 3.0 # Determines the efficacy of investment in generating increases in efficiency
mc = 5 # Marginal cost of production
c = 1 # Cost of a unit investment
P_ce = Uniform(x_el,x_eh);

## Equilibrium Price
With constant marginal cost and differentiate product market, the equilibrium price vector satisfy the following condition:
$$
p_j = \frac{mc}{1-\sigma_j}
$$
for $j = 1, \dots, N$. Moreover,
$$
\sigma_j = \frac{e^{g(\omega_j)}}{\sum\limits_i{e^{g(\omega_i)}}}
$$
where we define the utility function of consumers in the following way:
$$ e^{g(\omega)} = \mathbb{1}\{\omega\leq \omega^*\}e^\omega + \mathbb{1}\{\omega > \omega^*\}e^{\omega^*}[2-e^{-(\omega-\omega^*)}]
$$

Finally, we set $p^e(\omega = 3) = \delta$, so that a new entrant will enter at $\omega = 3$ with probability $\delta$ and at $\omega = 4$ with probability $1 - \delta$. Therefore, I code the function that computes the equilibrium prices given the state firms and marginal cost.

In [4]:
# Inputs: state of firms, marginal cost
# Output: Equilibrium prices
function prices(p)
    global s
    num = (Ω[s] .<= ω_star).*exp.(Ω[s] .- p) + 
         ( Ω[s] .> ω_star ).*exp.(ω_star .- p).*( 2 .- exp.( -(Ω[s] .- ω_star) ));
    den = 1 + sum( num );
    σ = num/den;
    equ = p .- mc./(1 .- σ);
    return equ
end

prices (generic function with 1 method)

In [5]:
# Proof of Functionality
N = 2
p = [2.0]
s = [4]
initial = p
# prices_temp(p) = prices(p,initial[N+1:2*N])
solution = nlsolve( prices, initial, autodiff = :forward)
p = solution.zero

1-element Array{Float64,1}:
 5.814553311936476

## Current Profit
Given any market structure and price profile, we compute the current profit of incumbents.

In [6]:
function current_profit(p)
    global s
    num = (Ω[s] .<= ω_star).*exp.(Ω[s] .- p) + 
        ( Ω[s] .> ω_star ).*exp.(ω_star .- p).*( 2 .- exp.( -(Ω[s] .- ω_star) ));
    den = 1 + sum( num );
    σ = num/den;
    return M*(p .- mc).*σ
end

current_profit (generic function with 1 method)

In [7]:
# Proof of Functionality
current_profit(p)

1-element Array{Float64,1}:
 0.5705486409656974

# One Incumbent

## Transition Kernel
To find the equilibrium policy function, we compute the transition kernel for a range of investment values. This is used compute the expected value function of the incumbent firm (just for monopolist's problem).

In [8]:
x = 0.5*(0:N_x - 1)/(N_x)
p_ν1 = a*x ./(1 .+ a*x)
p_ν = δ
p_τ = zeros(N_x,3)
p_τ[:,1] = p_ν1*(1 - p_ν)# probability of τ = 1
p_τ[:,2] = p_ν1*p_ν + (1 .- p_ν1)*(1 - p_ν) # probability of τ = 0
p_τ[:,3] = (1 .- p_ν1)*p_ν # probability of τ = -1;

## Initial Values

In [9]:
V0 = zeros(k+2 , 1)
V1 = Array{Float64,2}(undef, k, 1)
initial = [1.0]
x_index = ones(k , 1)
check = 1;

## Iteration with N = 1

In [10]:
# Fixed Point Loop
while check == 1
    V0_posi = V0[3:k+2] .* p_τ[:,1]'
    V0_zero = V0[2:k+1] .* p_τ[:,2]'
    V0_minu = V0[1:k]   .* p_τ[:,3]'
    EV = V0_minu + V0_zero + V0_posi
# Over state space
    for i = 1:k
        s = [i]
        solution = nlsolve( prices, initial, autodiff = :forward)
        p = solution.zero
        π_c = current_profit(p)
# Compute Value function at different investment (x) 
        temp = [Φ ; π_c .- c*x + β*EV[i,:]]
        max_V, ind = findmax( temp , dims = 1)
        x_index[i] = ind[1]
        V1[i] = max_V[1]
    end
    if norm( V1 - V0[2:k+1] ) < 0.001
        check = 0
    end
    V0[2:k+1] = V1
end
V_m = V1

21×1 Array{Float64,2}:
  0.1                
  0.14451898320512307
  0.38904604700212225
  1.5148887661752815 
  3.2399052423979464 
  4.818109615211219  
  6.1247663276818445 
  7.191850405742376  
  8.072263458251815  
  8.810405711952052  
  9.439243956366631  
  9.982652993666338  
 10.458032716101913  
 10.878265839092057  
 11.253081058913333  
 11.588961647736435  
 11.88994179966285   
 12.159638245684295  
 12.401292765166938  
 12.61781044996395   
 12.811794294996748  

The exit and investment decision is therefore

In [11]:
# Exit Decision: χ = 1 if the firm decides to stay in
χ_m = round.(Int, x_index) .!= 1
println(χ_m)
# Investment Decision: x_m is the investment of the monopolist
x_index_m = (χ_m .== 0).*1 + (χ_m .== 1).*(round.(Int, x_index) .- 1)
x_m = x[x_index_m]

Bool[0; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1; 1]


21×1 Array{Float64,2}:
 0.0   
 0.0   
 0.0625
 0.3   
 0.3875
 0.345 
 0.2825
 0.225 
 0.175 
 0.1325
 0.0975
 0.0675
 0.0425
 0.02  
 0.0025
 0.0   
 0.0   
 0.0   
 0.0   
 0.0   
 0.0   

I set the lower bound of productivity to 1 and upper bound to 21 and solve the fixed point problem for the monopolist to find the actual bounds that can be used later on for more complex industry structure. Here I find $\underline{\underline{\omega}}^m = 2$ and for $\bar\omega^m = 14$. Therefore, if $\omega < \underline{\underline{\omega}}^m$, the monopolist exits and if $\omega \geq \bar\omega^m$, the monopolist does not invest. These bounds are different than what is stated in the paper. Therefore, I used a wider state space to correct for possible mistakes in finding the bounds.

# Two Incumbents

There exists a $\underline{\underline{\omega}}$ such that $\mathcal{X}(\omega,s)=0$ whenever $\omega < \underline{\underline{\omega}}$. EP show that $\underline{\underline{\omega}}^m < \underline{\underline{\omega}}$ which means that if a market induces a monopolist to exit, it will induce induce exit when more than one firm can be active. Moreover, we take $\bar\omega^m + \bar\tau - 1$ as the upper bound to $\Omega$, where $\bar\tau$ is the upper bound to the increments in $\omega$ in a given period.

We come back to computational problem with $N = 2$ assuming that $\Omega = \{ \underline{\underline{\omega}}, \underline{\underline{\omega}} + 1, \dots , \bar\omega^m + \bar\tau - 1\} \equiv [2, 3, \dots, 14] \equiv [1, \dots, k]$. The Bellman equation for a firm whose $\omega = \omega_1$ and which has a single competitor with an $\omega = \omega_2$ is given by
$$
V\left[\omega_{1}, \omega_{2}\right]=\max \left\{\Phi, \sup _{\left(x_{1} \geq 0\right)} \pi\left(\omega_{1}, \omega_{2}\right)-c x +\beta \Sigma_{\tau_1, \tau_{2}, \nu} V\left[\omega_{1}+\tau_{1}+\nu, \omega_{2}+\tau_{2}+\nu\right] p\left[\tau_{1} | x_{1}, \nu\right] p\left[\tau_{2} | x_{2}, \nu\right] p(\nu)\right\}
$$

for $(\omega_1, \omega_2) \in \Omega^2$. I set $V^0(\omega_1,\omega_2)=l^*(\omega_1)$: the converged value function of monopolist's problem. We distinguish $(\omega_1 , \omega_2)$ points at which $V^{i-1}(\omega_2,\omega_1)>\Phi$ from the points at which this condition is not satisfied.

For the points at which there is an incumbent competitor, we use the following Kuhn-Tucker condition to estimate the $x^i$:

$\begin{aligned}
x^{i}\left\{\Sigma_{\tau_{1}, \tau_{2}, v} V^{i-1}\left[\omega_{1}+\tau_{1}+\nu, \omega_{2}+\tau_{2}+\nu\right] p\left[\tau_{2} | x^{i-1}\left(\omega_{2}, \omega_{1}\right), \nu\right]\left\{\partial p\left[\tau_{1} | x^{i}\left(\omega_{1}, \omega_{2}\right), \nu\right] / \partial x^{i}\right\} p(\nu)-c / \beta\right\}=0
    \end{aligned}$
where $x^{i}\left(\omega_{1}, \omega_{2}\right) \geq 0$. Moreover, in our problem $\partial p\left[\tau_{1} | x^{i}\left(\omega_{1}, \omega_{2}\right), \nu\right] / \partial x^{i}$ is analytically computed.
$$
\partial p\left[\nu_{1} | x^{i}\left(\omega_{1}, \omega_{2}\right), \nu\right] / \partial x^{i} = 
\left\{\mathbb{1}\{ \nu_{1} = 1\} - \mathbb{1}\{ \nu_{1} = 0\}\right\}\frac{a}{(1+ax)^2}
$$

!!! The function below is to continue the expected value of the value function given the industry structure. I should continue the probability function, then, compute the value function related to each probability.

## Investment Function with N = 2

The function dif(x) returns the residuals of equation (13). It uses the FOC of the value function of firms where $V^{i-1}(\omega_1,\omega_2) > \Phi$ ($\chi = 1$) to find the optimal investment.

Also, this function returns the the residuals of equation (13) modified for the situation where only one firm decides to stay in. It is used to find the optimal investment of firm 1 if $V^{i-1}(\omega_2,\omega_1) = \Phi$ ($\chi = 0$). First, I compute 

$$
V^{e i}\left(1, \omega_{1}\right)=\beta \Sigma_{\tau, \omega, \nu} V^{i-1}\left(\omega, \omega_{1}+\tau+\nu\right) p^{e}[\omega | \nu] p\left[\tau | x^{i-1}\left(\omega_{1}, \omega_{2}\right), \nu\right] p(\nu)
$$

We know the probability distribution $p(\nu)$. Then we should compute $p\left[\tau | x^{i-1}\left(\omega_{1}, \omega_{2}\right), \nu\right]$. Finally, we go back to use equation (13) to find firm 1's investment.

In [12]:
function dif(x)
    global entrant, s, χ0 , x0, V0, p_e
    s1 = s[1]
    s2 = s[2]
    ω1_pointer = (s2 - 1)*(k+1) + s1
    ω2_pointer = (s1 - 1)*(k+1) + s2
    
    # probabilities and partial probabilities
    p_ν1 = a*x0[ω1_pointer]/(1.0 .+ a*x0[ω1_pointer]) #using x^(j-1)
    p_ν2 = a*x0[ω2_pointer]/(1.0 .+ a*x0[ω2_pointer])
    par_p_ν1 = a/(1.0 + a*x[1])^2 #using x^j
    
    # initial expected values
    EV_e = .0
    EV = .0
    
    # If ω2 decides to exit at j-1
    if χ0[ω2_pointer] == 0
        for i = 1:2 ,j = 1:2
            τ1 = ν1[j] - ν[i]
            pointer = (s1 + τ1 + ν[i] - 1)*(k+1) + s_e - ν[i]
            prob = ( (1 - ν[i])*(1 - δ) + ν[i]*δ )*
            ( ( 1 - ν1[j] )*(1 - p_ν1) + ν1[j]*p_ν1)
            EV_e = EV_e + β*prob*V0[pointer]
        end
        #smoothing the entry decision
        p_e = cdf(P_ce, EV_e[1]) 
        for i = 1:2, j = 1:2, l = 1:2
            τ1 = ν1[j] - ν[i]
            pointer = (e[l]*(s_e - ν[i]) + (1- e[l])*1 - 1)*(k+1) + s1 + τ1 + ν[i]
            prob = ( (1 - ν[i])*(1 - δ) + ν[i]*δ )*
            ((1 - e[l])*(1 - p_e) + e[l]*p_e ) *
            ( ( 1 - ν1[j] )*(-par_p_ν1) + ν1[j]*par_p_ν1)
            EV = EV + prob*V0[pointer]

        end
    # If ω2 decides to stay at j-1
    elseif χ0[ω2_pointer] == 1
        for i = 1:2, j = 1:2, l = 1:2 
            τ2 = ν2[l] - ν[i]
            τ1 = ν1[j] - ν[i]
            pointer = (s2 + τ2 + ν[i] - 1)*(k+1) + s1 + τ1 + ν[i]
            prob = ( (1 - ν[i])*(1 - δ) + ν[i]*δ )* 
            ( ( 1 - ν2[l] )*(1 - p_ν2) + ν2[l]*p_ν2)* 
            ( ( 1 - ν1[j] )*(-par_p_ν1) + ν1[j]*par_p_ν1)
            EV = EV + prob*V0[pointer]
        end
    end
    return EV .- c/β
end

dif (generic function with 1 method)

## Initial Values  and Iteration with N = 2

In [13]:
k = 15
V0 = repeat(V_m[1:k+1], k+1, 1)
x0 = repeat(x_m[1:k+1], k+1, 1)
χ0 = repeat(χ_m[1:k+1], k+1, 1)
V1 = copy(V0)
V_check = copy(V0)
x1 = copy(x0)
χ1 = copy(χ0)
check = 0
initial_x = [0.1]
initial_p = [1.0; 1.0];

In [14]:
iter = 1
#fixed point loop
while iter < 201 && check == 0
    for mkt_block = 1:k, mkt_row = 2:k
        mkt_count = (mkt_block-1)*(k+1) + mkt_row
        s = [mkt_row ; mkt_block ]
        s1 = s[1]
        s2 = s[2]
        ω1_pointer = (s2 - 1)*(k+1) + s1
        ω2_pointer = (s1 - 1)*(k+1) + s2
        
        solution = nlsolve( dif, initial_x , autodiff = :forward) #this function also returns entrant
        x_u = (solution.zero .>= 0).* solution.zero + (solution.zero .< 0)*0
        if solution.f_converged == 0
            x_u[1] = 0.0
#             println("Iter=",iter, "S=",s)
#             println("x_u=",solution)
        end

        
#         println("s=",s, ", x_u=",x_u)
        p_ν1 = a*x_u[1]/(1.0 .+ a*x_u[1])
        p_ν2 = a*x0[ω2_pointer]/(1.0 .+ a*x0[ω2_pointer])
        EV = .0
        
        # PM Page 567: where it is understood that if ... π(ω_1) for π(ω_1,ω_2)
        if χ0[ω2_pointer] == 0
            s = [s1]
            initial_p = [1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)
            for i = 1:2 , j = 1:2, l = 1:2
                τ1 = ν1[j] - ν[i]
                pointer = (e[l]*(s_e - ν[i]) + (1- e[l])*1 - 1)*(k+1) + s1 + τ1 + ν[i]
                prob = ( (1 - ν[i])*(1 - δ) + ν[i]*δ )* 
                ((1 - e[l])*(1 - p_e) + e[l]*p_e ) *
                ( ( 1 - ν1[j] )*(1 - p_ν1) + ν1[j]*p_ν1)
                EV = EV .+ prob*V0[pointer]
            end
        elseif χ0[ω2_pointer] == 1
            initial_p = [1.0; 1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)
            for i = 1:2, j = 1:2, l = 1:2
                τ1 = ν1[j] - ν[i]
                τ2 = ν2[l] - ν[i]
                pointer = (s2 + τ2 + ν[i] - 1)*(k+1) + s1 + τ1 + ν[i]
                prob = ( (1 - ν[i])*(1 - δ) + ν[i]*δ )*
                ( ( 1 - ν2[l] )*(1 - p_ν2) + ν2[l]*p_ν2)* 
                ( ( 1 - ν1[j] )*(1 - p_ν1) + ν1[j]*p_ν1)
                EV = EV .+ prob*V0[pointer]
            end
        end

        max_V, ind = findmax( [Φ ; π_c[1] - c*x_u[1] + β*EV[1] ])
        V1[mkt_count] = max_V
        x1[mkt_count] = (ind == 2)*x_u[1]
        χ1[mkt_count] = (ind == 2)
        V_check[mkt_count] = max_V            
    end
    if norm( V_check - V0 )/(1 + norm(V_check)) < 1e-3
        check = 1
        println( "iter: ", iter, " Percentage: ", norm( V_check - V0 )/(1 + norm(V_check)) )
        println(iter)
    end
    if iter%10 == 0
        println( "iter: ", iter, " Percentage: ", norm( V_check - V0 )/(1 + norm(V_check)) )
        println( "iter: ", iter, " Norm: ", norm( V_check - V0 ) )
    end
    V0 = copy(V1)
    x0 = copy(x1)
    χ0 = copy(χ1)

    iter += 1
end
V_d = V0
x_d = x0
χ_d = χ0;

iter: 10 Percentage: 0.017655559221703806
iter: 10 Norm: 2.6408400313388776
iter: 20 Percentage: 0.006031382661611949
iter: 20 Norm: 0.968767240757164
iter: 30 Percentage: 0.002560717800514416
iter: 30 Norm: 0.4239197096716431
iter: 40 Percentage: 0.0011440207742087038
iter: 40 Norm: 0.19196433096196072
iter: 42 Percentage: 0.0009758976558138809
42


In [15]:
[x1[1:k+1] V1[1:k+1] x1[(k+1)+1:2*(k+1)] V1[(k+1)+1:2*(k+1)]]

16×4 Array{Float64,2}:
 0.0         0.1      0.0         0.1    
 0.387024    5.90601  0.394712    5.9707 
 0.398107    7.62488  0.406075    7.71821
 0.296308    9.41269  0.30177     9.5182 
 0.131907   10.7319   0.134662   10.8414 
 0.0323208  11.4447   0.0339782  11.5575 
 0.0        11.8826   0.0        11.9979 
 0.0        12.0752   0.0        12.1918 
 0.0        12.1622   0.0        12.2793 
 0.0        12.2086   0.0        12.3259 
 0.0        12.2386   0.0        12.356  
 0.0        12.2612   0.0        12.3786 
 0.0        12.2799   0.0        12.3973 
 0.0        12.2961   0.0        12.4135 
 0.0        12.3104   0.0        12.4279 
 0.0        11.589    0.0        11.589  

# Two Incumbents: Modified for Cycles

## Initial Values and Iteration with N = 2

In [16]:
k = 15
V0 = repeat(V_m[1:k+1], k+1, 1)
x0 = repeat(x_m[1:k+1], k+1, 1)
χ0 = repeat(χ_m[1:k+1], k+1, 1)
V1 = copy(V0)
V_check = copy(V0)
x1 = copy(x0)
χ1 = copy(χ0)
check = 0
initial_x = [0.1]
initial_p = [1.0; 1.0];

In [17]:
iter = 1
#fixed point loop
while iter < 201 && check == 0
    V_check = copy(V0)
    for mkt_block = 1:k, mkt_row= 2:mkt_block
        mkt_count = (mkt_block-1)*(k+1) + mkt_row
        s = [mkt_row ; mkt_block ]
#         println("s=",s)
        s1 = s[1]
        s2 = s[2]
        ω1_pointer = (s2 - 1)*(k+1) + s1
        ω2_pointer = (s1 - 1)*(k+1) + s2

        solution = nlsolve( dif, initial_x , autodiff = :forward ) #this function also returns p_e
        x_u = (solution.zero .>= 0).* solution.zero + (solution.zero .< 0)*0
        x_u = (solution.f_converged == 1)*x_u[1] + (solution.f_converged == 0)*0
        p_ν1 = a*x_u[1]/(1.0 .+ a*x_u[1])
        p_ν2 = a*x0[ω2_pointer]/(1.0 .+ a*x0[ω2_pointer])
        EV = .0

        # PM Page 567: where it is understood that if ... π(ω_1) for π(ω_1,ω_2)
        if χ0[ω2_pointer] == 0
            s = [s1]
            initial_p = [.1]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)
            for i = 1:2 , j = 1:2, l = 1:2
                τ1 = ν1[j] - ν[i]
                pointer = (e[l]*(s_e-ν[i]) + (1- e[l])*1 - 1)*(k+1) + s1+τ1+ν[i]
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )* 
                ((1-e[l])*(1-p_e) + e[l]*p_e )*
                ( ( 1-ν1[j] )*(1-p_ν1) + ν1[j]*p_ν1)
                EV = EV .+ prob*V0[pointer]
            end
        elseif χ0[ω2_pointer] == 1
            initial_p = [.1; .1]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)
            for i = 1:2, j = 1:2, l = 1:2
                τ1 = ν1[j] - ν[i]
                τ2 = ν2[l] - ν[i]
                pointer = (s2+τ2+ν[i]-1)*(k+1) + s1+τ1+ν[i]
                prob = ( (1 - ν[i])*(1 - δ) + ν[i]*δ )*
                ( ( 1 - ν2[l] )*(1 - p_ν2) + ν2[l]*p_ν2)* 
                ( ( 1 - ν1[j] )*(1 - p_ν1) + ν1[j]*p_ν1)
                EV = EV .+ prob*V0[pointer]
            end
        end
        max_V, ind = findmax( [Φ ; π_c[1] - c*x_u[1] + β*EV[1]])
        V1[mkt_count] = max_V
        x1[mkt_count] = (ind == 2)*x_u[1]
        χ1[mkt_count] = (ind == 2)
    end
    # Update value function of those with lower quality
    V0 = copy(V1)
    x0 = copy(x1)
    χ0 = copy(χ1)
    for mkt_block = 1:k, mkt_row= k:-1:mkt_block+1
        mkt_count = (mkt_block-1)*(k+1) + mkt_row
        s = [mkt_row ; mkt_block ]
        s1 = s[1]
        s2 = s[2]
        ω1_pointer = (s2 - 1)*(k+1) + s1
        ω2_pointer = (s1 - 1)*(k+1) + s2

        solution = nlsolve( dif, initial_x , autodiff = :forward ) #this function also returns entrant
        x_u = (solution.zero .>= 0).* solution.zero + (solution.zero .< 0)*0
        x_u = (solution.f_converged == 1)*x_u[1] + (solution.f_converged == 0)*0
        p_ν1 = a*x_u[1]/(1.0 .+ a*x_u[1])
        p_ν2 = a*x0[ω2_pointer]/(1.0 .+ a*x0[ω2_pointer])
        EV = .0

        # PM Page 567: where it is understood that if ... π(ω_1) for π(ω_1,ω_2)
        if χ0[ω2_pointer] == 0
            s = [s1]
            initial_p = [.1]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)
            for i = 1:2 , j = 1:2, l = 1:2
                τ1 = ν1[j] - ν[i]
                pointer = (e[l]*(s_e-ν[i]) + (1- e[l])*1 - 1)*(k+1) + s1+τ1+ν[i]
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )* 
                ((1-e[l])*(1-p_e) + e[l]*p_e )*
                ( ( 1-ν1[j] )*(1-p_ν1) + ν1[j]*p_ν1)
                EV = EV .+ prob*V0[pointer]
            end
        elseif χ0[ω2_pointer] == 1
            initial_p = [.1; .1]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)
            for i = 1:2, j = 1:2, l = 1:2
                τ1 = ν1[j] - ν[i]
                τ2 = ν2[l] - ν[i]
                pointer = (s2+τ2+ν[i]-1)*(k+1) + s1+τ1+ν[i]
                prob = ( (1 - ν[i])*(1 - δ) + ν[i]*δ )*
                ( ( 1 - ν2[l] )*(1 - p_ν2) + ν2[l]*p_ν2)* 
                ( ( 1 - ν1[j] )*(1 - p_ν1) + ν1[j]*p_ν1)
                EV = EV .+ prob*V0[pointer]
            end
        end
        max_V, ind = findmax( [Φ ; π_c[1] - c*x_u[1] + β*EV[1]])
        V1[mkt_count] = max_V
        x1[mkt_count] = (ind == 2)*x_u[1]
        χ1[mkt_count] = (ind == 2)
    end
    
    # s = (1,1) is included in the loop above. It shouldn't
    V1[1] = Φ
    x1[1] = 0.0
    χ1[1] = 0
    
    if norm( V1 - V_check )/(1 + norm(V1)) < 1e-3
        check = 1
        println( "iter: ", iter, " Percentage: ", norm( V1 - V_check )/(1 + norm(V1)) )
        println( "iter: ", iter, " Norm: ", norm( V1 - V_check ) )

    end
    if iter%10 == 0
        println( "iter: ", iter, " Percentage: ", norm( V1 - V_check )/(1 + norm(V1)) )
        println( "iter: ", iter, " Norm: ", norm( V1 - V_check ) )
    end
    V0 = copy(V1)
    x0 = copy(x1)
    χ0 = copy(χ1)
    iter += 1
end
V_d = V0
x_d = x0
χ_d = χ0;

iter: 10 Percentage: 0.017558078297795775
iter: 10 Norm: 2.632410698171077
iter: 20 Percentage: 0.005882737619384937
iter: 20 Norm: 0.9464128232519438
iter: 30 Percentage: 0.0024722948569894914
iter: 30 Norm: 0.4096072425974339
iter: 40 Percentage: 0.0011026801491425702
iter: 40 Norm: 0.1850954722849914
iter: 42 Percentage: 0.000940571504256729
iter: 42 Norm: 0.15813441896762268


256×1 BitArray{2}:
 0
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

In [18]:
[x1[1:k+1] V1[1:k+1] x1[(k+1)+1:2*(k+1)] V1[(k+1)+1:2*(k+1)]]

16×4 Array{Float64,2}:
 0.0         0.1      0.0        0.1    
 0.387774    5.93367  0.395945   5.98995
 0.397504    7.65259  0.40514    7.74443
 0.295376    9.43754  0.301144   9.54144
 0.131      10.7541   0.13398   10.8629 
 0.0311042  11.4649   0.032915  11.5774 
 0.0        11.9004   0.0       12.0157 
 0.0        12.0878   0.0       12.2044 
 0.0        12.1689   0.0       12.286  
 0.0        12.211    0.0       12.3284 
 0.0        12.2392   0.0       12.3566 
 0.0        12.2614   0.0       12.3789 
 0.0        12.2801   0.0       12.3975 
 0.0        12.2963   0.0       12.4137 
 0.0        12.3106   0.0       12.4281 
 0.0        11.589    0.0       11.589  

# Three Incumbents

## Investment Function with N = 3
The logic behind this investment function is same as the investment function in a market with two incumbents. Here we have to consider more cases than a market with two incumbents. Specifically, in a market with three incumbent firms, a firm might face an exit decision from one, two or none of its competitors. Moreover, each incumbent faces the threat of entry from at most one entrant in each period.

In [19]:
function tif(x)
    global s, χ0 , x0, V0, p_e
    p = a*x0[address]./(1.0 .+ a*x0[address]) #ν1, ν2, ν3
    par_p_ν1 = a/(1.0 + a*x[1])^2
    EV = .0
    EV_e = .0
    if χ0[address[2]] == 1 && χ0[address[3]] == 1
        for i=1:2,j=1:2, l=1:2, m=1:2
            τ[1,j] = ν[j] - ν[i]
            τ[2,l] = ν[l] - ν[i]
            τ[3,m] = ν[m] - ν[i]
            pointer = [s1+τ[1,j]+ν[i] s2+τ[2,l]+ν[i]-1  s3+τ[3,m]+ν[i]-1]*k_plus
            pointer = round(Int, pointer[1])
            prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
            ((1-ν[m])*(1-p[3]) + (ν[m]*p[3]))*
            ((1-ν[l])*(1-p[2]) + (ν[l]*p[2]))*
            ((1-ν[j])*(-par_p_ν1) + ν[j]*par_p_ν1)
            EV += prob*V0[pointer[1]]
        end
    # we have an entrant and at most one entrant = x_e(2) = ∞
    elseif χ0[address[2]] == 1 && χ0[address[3]] == 0
        for i=1:2,j=1:2,l=1:2
            τ[1,j] = ν[j] - ν[i]
            τ[2,l] = ν[l] - ν[i]
            pointer = [s_e-ν[i] s1+τ[1,j]+ν[i]-1 s2+τ[2,l]+ν[i]-1]*k_plus
            pointer = round(Int, pointer[1])
            prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
            ((1-ν[l])*(1-p[2]) + (ν[l]*p[2]))*
            ((1-ν[j])*(1-p[1]) + ν[j]*p[1])
            EV_e += prob*V0[pointer[1]]
        end
        p_e = cdf(P_ce, EV_e[1])
        for i=1:2,j=1:2,l=1:2, m=1:2
            τ[1,j] = ν[j] - ν[i]
            τ[2,l] = ν[l] - ν[i]
            pointer = [s1+τ[1,j]+ν[i] s2+τ[2,l]+ν[i]-1 e[m]*(s_e-ν[i])+(1-e[m])*1-1]*k_plus
            pointer = round(Int, pointer[1])
            prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
            ((1-e[m])*(1-p_e) + e[m]*p_e)*
            ((1-ν[l])*(1-p[2]) + (ν[l]*p[2]))*
            ((1-ν[j])*(-par_p_ν1) + ν[j]*par_p_ν1)
            EV += prob*V0[pointer[1]]
        end
    elseif χ0[address[2]] == 0 && χ0[address[3]] == 1
        for i=1:2,j=1:2,l=1:2
            τ[1,j] = ν[j] - ν[i]
            τ[3,l] = ν[l] - ν[i]
            pointer = [s_e-ν[i] s1+τ[1,j]+ν[i]-1 s3+τ[3,l]+ν[i]-1]*k_plus
            pointer = round(Int, pointer[1])
            prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
            ((1-ν[l])*(1-p[3]) + (ν[l]*p[3]))*
            ((1-ν[j])*(1-p[1]) + ν[j]*p[1])
            EV_e += prob*V0[pointer[1]]
        end
        p_e = cdf(P_ce, EV_e[1])
        for i=1:2,j=1:2,l=1:2, m=1:2
            τ[1,j] = ν[j] - ν[i]
            τ[3,l] = ν[l] - ν[i]
            pointer = [s1+τ[1,j]+ν[i] s3+τ[3,l]+ν[i]-1 e[m]*(s_e-ν[i])+(1-e[m])*1-1]*k_plus
            pointer = round(Int, pointer[1])
            prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
            ((1-e[m])*(1-p_e) + e[m]*p_e)*
            ((1-ν[l])*(1-p[3]) + (ν[l]*p[3]))*
            ((1-ν[j])*(-par_p_ν1) + ν[j]*par_p_ν1)
            EV += prob*V0[pointer[1]]
        end
    elseif χ0[address[2]] == 0 && χ0[address[3]] == 0
        for i=1:2,j=1:2
            τ[1,j] = ν[j] - ν[i]
            pointer = [s_e-ν[i] s1+τ[1,j]+ν[i]-1 0]*k_plus
            pointer = round(Int, pointer[1])
            prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
            ((1-ν[j])*(1-p[1]) + ν[j]*p[1])
            EV_e += prob*V0[pointer[1]]
        end
        p_e = cdf(P_ce, EV_e[1])
        for i=1:2,j=1:2, m=1:2
            τ[1,j] = ν[j] - ν[i]
            pointer = [s1+τ[1,j]+ν[i] e[m]*(s_e-ν[i])+(1-e[m])*1-1 0]*k_plus
            pointer = round(Int, pointer[1])
            prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
            ((1-e[m])*(1-p_e) + e[m]*p_e)*
            ((1-ν[j])*(-par_p_ν1) + ν[j]*par_p_ν1)
            EV += prob*V0[pointer[1]]
        end
    end
    return EV[1] - c/β
end

tif (generic function with 1 method)

## Initial Values and Iteration with N = 3

In [20]:
k_plus = [1 ; k + 1 ; (k+1)^2]
s = [3,2,1]
s1 = s[1]
s2 = s[2]
s3 = s[3]

ω = [s1  s2-1 s3-1;
    s2 s1-1 s3-1;
    s3 s1-1 s2-1]
address = round.(Int, ω*k_plus)

# Initial Values, policy function
V0 = repeat( V_d , k+1, 1)
x0 = repeat( x_d , k+1, 1) 
χ0 = repeat( χ_d , k+1, 1) 
V1 = copy(V0)
V_check = copy(V0)
x1 = copy(x0)
χ1 = copy(χ0)
check = 0
iter = 1
initial_x = [0.1]
initial_p = [1.0; 1.0; 1.0];

In [21]:
iter = 1
check = 0
#fixed point loop
while iter < 201 && check == 0
    for ω1 =2:k, ω2 =1:k, ω3=1:ω2
        # set up
        s = [ω1; ω2; ω3]
        s1 = s[1]
        s2 = s[2]
        s3 = s[3]
        ω = [s1 s2-1 s3-1;
            s2 s1-1 s3-1;
            s3 s1-1 s2-1]
        address = round.(Int, ω*k_plus)
        ω1_permutation = [s1 s2-1 s3-1;
                        s1 s3-1 s2-1]
        permutation = round.(Int, ω1_permutation*k_plus)

        # investment, probability of increase in productivity, j-1 probability of increase of all firms
        solution = nlsolve( tif, initial_x , autodiff = :forward) #this function also returns p_e
        x_u = (solution.zero .>= 0).* solution.zero + (solution.zero .< 0)*0
        x_u = (solution.f_converged == 1)*x_u[1] + (solution.f_converged == 0)*0        
        p_ν1 = a*x_u[1]/(1.0 .+ a*x_u[1])
        pr = a*x0[address]./(1.0 .+ a*x0[address]) #ν1, ν2, ν3
        EV = .0
#         println(s,χ0[address[2:end]])
        # different regions of exit policy function
        if χ0[address[2]] == 1 && χ0[address[3]] == 1
            initial_p = [1.0; 1.0; 1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)
            for i=1:2,j=1:2, l=1:2, m=1:2
                τ[1,j] = ν[j] - ν[i]
                τ[2,l] = ν[l] - ν[i]
                τ[3,m] = ν[m] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] s2+τ[2,l]+ν[i]-1  s3+τ[3,m]+ν[i]-1]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-ν[m])*(1-pr[3]) + ν[m]*pr[3])*
                ((1-ν[l])*(1-pr[2]) + ν[l]*pr[2])*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        elseif χ0[address[2]] == 1 && χ0[address[3]] == 0
            s = [s1; s2]
    #         println("s = ", s)
            initial_p = [1.0; 1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)
            for i=1:2,j=1:2,l=1:2, m=1:2
                τ[1,j] = ν[j] - ν[i]
                τ[2,l] = ν[l] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] s2+τ[2,l]+ν[i]-1 e[m]*(s_e-ν[i])+(1-e[m])*1-1]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-e[m])*(1-p_e) + e[m]*p_e)*
                ((1-ν[l])*(1-pr[2]) + ν[l]*pr[2])*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        elseif χ0[address[2]] == 0 && χ0[address[3]] == 1
            s = [s1; s3]
            initial_p = [1.0; 1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)        
            for i=1:2,j=1:2,l=1:2, m=1:2
                τ[1,j] = ν[j] - ν[i]
                τ[3,l] = ν[l] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] s3+τ[3,l]+ν[i]-1 e[m]*(s_e-ν[i])+(1-e[m])*1-1]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-e[m])*(1-p_e) + e[m]*p_e)*
                ((1-ν[l])*(1-pr[3]) + (ν[l]*pr[3]))*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        elseif χ0[address[2]] == 0 && χ0[address[3]] == 0
            s = [s1]
            initial_p = [1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)        
            for i=1:2,j=1:2, m=1:2
                τ[1,j] = ν[j] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] e[m]*(s_e-ν[i])+(1-e[m])*1-1 0]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-e[m])*(1-p_e) + e[m]*p_e)*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        end
        max_V, ind = findmax( [Φ ; π_c[1] - c*x_u[1] + β*EV[1] ])
        V1[permutation] .= max_V
        x1[permutation] .= (ind == 2)*x_u[1]
        χ1[permutation] .= (ind == 2)
        V_check[permutation] .= max_V
    end

    if norm( V1 - V0 )/(1 + norm(V1)) < 1e-3
        check = 1
        println( "iter: ", iter, " Percentage: ", norm( V1 - V0 )/(1 + norm(V1)) )
    end
    if iter%10 == 0
        println( "iter: ", iter, " Percentage: ", norm( V1 - V0 )/(1 + norm(V1)) )
        println( "iter: ", iter, " Norm: ", norm( V1 - V0 ) )
    end
    V0 = copy(V1)
    x0 = copy(x1)
    χ0 = copy(χ1)
    iter += 1
end
V_t = V0
x_t = x0
χ_t = χ0;

iter: 10 Percentage: 0.01863414447903435
iter: 10 Norm: 11.419191590281162
iter: 20 Percentage: 0.009696021464691403
iter: 20 Norm: 5.673279538750325
iter: 30 Percentage: 0.003215007646670667
iter: 30 Norm: 1.8468076112962235
iter: 40 Percentage: 0.0010906810980968052
iter: 40 Norm: 0.621609440323768
iter: 50 Percentage: 0.001164697529999304
iter: 50 Norm: 0.6608858925236841
iter: 53 Percentage: 0.0009977826439710174


In [22]:
[x1[1:k+1] V1[1:k+1] x1[(k+1)+1:2*(k+1)] V1[(k+1)+1:2*(k+1)]]

16×4 Array{Float64,2}:
 0.0          0.1      0.0          0.1    
 0.354956     4.7037   0.355689     4.7215 
 0.378128     6.3119   0.375836     6.32004
 0.284433     8.03906  0.282264     8.02539
 0.123129     9.32778  0.122538     9.30029
 0.0211353   10.0213   0.0212065    9.99001
 0.0         10.4367   0.0         10.4048 
 0.0         10.592    0.0         10.56   
 0.0         10.649    0.0         10.6169 
 0.0         10.6701   0.0         10.638  
 0.0         10.6781   0.0         10.646  
 0.0         10.6812   0.0         10.6491 
 0.0         10.6902   0.0         10.6591 
 0.0627848   11.0042   0.0593772   10.9644 
 0.00505999  11.4632   0.00433187  11.4275 
 0.0         11.589    0.0         11.589  

## Three Incumbents: Modified for Cycles

## Initial Values and Iteration with N = 3

In [23]:
k_plus = [1 ; k + 1 ; (k+1)^2]
s = [3,2,1]
s1 = s[1]
s2 = s[2]
s3 = s[3]

ω = [s1  s2-1 s3-1;
    s2 s1-1 s3-1;
    s3 s1-1 s2-1]
address = round.(Int, ω*k_plus)

# Initial Values, policy function
V0 = repeat( V_d , k+1, 1)
x0 = repeat( x_d , k+1, 1) 
χ0 = repeat( χ_d , k+1, 1) 
V1 = copy(V0)
V_check = copy(V0)
x1 = copy(x0)
χ1 = copy(χ0)
check = 0
iter = 1
initial_x = [0.1]
initial_p = [1.0; 1.0; 1.0];

In [24]:
iter = 1
#fixed point loop
while iter < 2 && check == 0
    V_check = copy(V0)
    for ω1 =2:k, ω2 =k:-1:ω1, ω3=1:ω2
        # set up
        s = [ω1; ω2; ω3]
    #     println("s = ", s)
        s1 = s[1]
        s2 = s[2]
        s3 = s[3]
        ω = [s1 s2-1 s3-1;
            s2 s1-1 s3-1;
            s3 s1-1 s2-1]
        address = round.(Int, ω*k_plus)
        ω1_permutation = [s1 s2-1 s3-1;
                        s1 s3-1 s2-1]
        permutation = round.(Int, ω1_permutation*k_plus)

        # investment, probability of increase in productivity, j-1 probability of increase of all firms
        solution = nlsolve( tif, initial_x , autodiff = :forward) #this function also returns p_e
        x_u = (solution.zero .>= 0).* solution.zero + (solution.zero .< 0)*0
        x_u = (solution.f_converged == 1)*x_u[1] + (solution.f_converged == 0)*0  
        p_ν1 = a*x_u[1]/(1.0 .+ a*x_u[1])
        pr = a*x0[address]./(1.0 .+ a*x0[address]) #ν1, ν2, ν3
        EV = .0

        # different regions of exit policy function
        if χ0[address[2:end]] == [1;1]
            initial_p = [1.0; 1.0; 1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)
            for i=1:2,j=1:2, l=1:2, m=1:2
                τ[1,j] = ν[j] - ν[i]
                τ[2,l] = ν[l] - ν[i]
                τ[3,m] = ν[m] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] s2+τ[2,l]+ν[i]-1  s3+τ[3,m]+ν[i]-1]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-ν[m])*(1-pr[3]) + ν[m]*pr[3])*
                ((1-ν[l])*(1-pr[2]) + ν[l]*pr[2])*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        elseif χ0[address[2]] == 1 && χ0[address[3]] == 0
            s = [s1; s2]
    #         println("s = ", s)
            initial_p = [1.0; 1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)
            for i=1:2,j=1:2,l=1:2, m=1:2
                τ[1,j] = ν[j] - ν[i]
                τ[2,l] = ν[l] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] s2+τ[2,l]+ν[i]-1 e[m]*(s_e-ν[i])+(1-e[m])*1-1]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-e[m])*(1-p_e) + e[m]*p_e)*
                ((1-ν[l])*(1-pr[2]) + ν[l]*pr[2])*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        elseif χ0[address[2]] == 0 && χ0[address[3]] == 1
            s = [s1; s3]
            initial_p = [1.0; 1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)        
            for i=1:2,j=1:2,l=1:2, m=1:2
                τ[1,j] = ν[j] - ν[i]
                τ[3,l] = ν[l] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] s3+τ[3,l]+ν[i]-1 e[m]*(s_e-ν[i])+(1-e[m])*1-1]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-e[m])*(1-p_e) + e[m]*p_e)*
                ((1-ν[l])*(1-pr[3]) + (ν[l]*pr[3]))*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        elseif χ0[address[2]] == 0 && χ0[address[3]] == 0
            s = [s1]
            initial_p = [1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)        
            for i=1:2,j=1:2, m=1:2
                τ[1,j] = ν[j] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] e[m]*(s_e-ν[i])+(1-e[m])*1-1 0]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-e[m])*(1-p_e) + e[m]*p_e)*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        end
        max_V, ind = findmax( [Φ ; π_c[1] - c*x_u[1] + β*EV[1] ])
        V1[permutation] .= max_V
        x1[permutation] .= (ind == 2)*x_u[1]
        χ1[permutation] .= (ind == 2)
    end
    # Update value function of those with lower quality
    V0 = copy(V1)
    x0 = copy(x1)
    χ0 = copy(χ1)
    
    for ω1 =2:k, ω2 =1:ω1, ω3=1:ω2
        # set up
        s = [ω1; ω2; ω3]
        s1 = s[1]
        s2 = s[2]
        s3 = s[3]
        ω = [s1 s2-1 s3-1;
            s2 s1-1 s3-1;
            s3 s1-1 s2-1]
        address = round.(Int, ω*k_plus)
        ω1_permutation = [s1 s2-1 s3-1;
                        s1 s3-1 s2-1]
        permutation = round.(Int, ω1_permutation*k_plus)

        # investment, probability of increase in productivity, j-1 probability of increase of all firms
        solution = nlsolve( tif, initial_x , autodiff = :forward) #this function also returns p_e
        x_u = (solution.zero .>= 0).* solution.zero + (solution.zero .< 0)*0
        x_u = (solution.f_converged == 1)*x_u[1] + (solution.f_converged == 0)*0  
        p_ν1 = a*x_u[1]/(1.0 .+ a*x_u[1])
        pr = a*x0[address]./(1.0 .+ a*x0[address]) #ν1, ν2, ν3
        EV = .0

        # different regions of exit policy function
        if χ0[address[2]] == 1 && χ0[address[3]] == 1
            initial_p = [1.0; 1.0; 1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)
            for i=1:2,j=1:2, l=1:2, m=1:2
                τ[1,j] = ν[j] - ν[i]
                τ[2,l] = ν[l] - ν[i]
                τ[3,m] = ν[m] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] s2+τ[2,l]+ν[i]-1  s3+τ[3,m]+ν[i]-1]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-ν[m])*(1-pr[3]) + ν[m]*pr[3])*
                ((1-ν[l])*(1-pr[2]) + ν[l]*pr[2])*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        elseif χ0[address[2]] == 1 && χ0[address[3]] == 0
            s = [s1; s2]
            initial_p = [1.0; 1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)
            for i=1:2,j=1:2,l=1:2, m=1:2
                τ[1,j] = ν[j] - ν[i]
                τ[2,l] = ν[l] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] s2+τ[2,l]+ν[i]-1 e[m]*(s_e-ν[i])+(1-e[m])*1-1]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-e[m])*(1-p_e) + e[m]*p_e)*
                ((1-ν[l])*(1-pr[2]) + ν[l]*pr[2])*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        elseif χ0[address[2]] == 0 && χ0[address[3]] == 1
            s = [s1; s3]
            initial_p = [1.0; 1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)        
            for i=1:2,j=1:2,l=1:2, m=1:2
                τ[1,j] = ν[j] - ν[i]
                τ[3,l] = ν[l] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] s3+τ[3,l]+ν[i]-1 e[m]*(s_e-ν[i])+(1-e[m])*1-1]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-e[m])*(1-p_e) + e[m]*p_e)*
                ((1-ν[l])*(1-pr[3]) + (ν[l]*pr[3]))*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        elseif χ0[address[2]] == 0 && χ0[address[3]] == 0
            s = [s1]
            initial_p = [1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)        
            for i=1:2,j=1:2, m=1:2
                τ[1,j] = ν[j] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] e[m]*(s_e-ν[i])+(1-e[m])*1-1 0]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-e[m])*(1-p_e) + e[m]*p_e)*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        end
        max_V, ind = findmax( [Φ ; π_c[1] - c*x_u[1] + β*EV[1] ])
        V1[permutation] .= max_V
        x1[permutation] .= (ind == 2)*x_u[1]
        χ1[permutation] .= (ind == 2)        
    end

    if norm( V1 - V_check )/(1 + norm(V1)) < 1e-3
        check = 1
        println( "iter: ", iter, " Percentage: ", norm( V1 - V_check )/(1 + norm(V1)) )
    end
    if iter%10 == 0
        println( "iter: ", iter, " Percentage: ", norm( V1 - V_check )/(1 + norm(V1)) )
        println( "iter: ", iter, " Norm: ", norm( V1 - V_check ) )
    end
    V0 = copy(V1)
    x0 = copy(x1)
    χ0 = copy(χ1)
    iter += 1
end
V_t = V0
x_t = x0
χ_t = χ0;

In [25]:
[x1[1:k+1] V1[1:k+1] x1[(k+1)+1:2*(k+1)] V1[(k+1)+1:2*(k+1)]]

16×4 Array{Float64,2}:
 0.0         0.1      0.0         0.1    
 0.387711    5.94538  0.385737    5.93528
 0.397545    7.66428  0.410803    7.67569
 0.295441    9.44943  0.319196    9.51743
 0.131058   10.7662   0.134021   10.875  
 0.0311655  11.4771   0.0329653  11.5896 
 0.0        11.9127   0.0        12.028  
 0.0        12.0989   0.0        12.2156 
 0.0        12.1787   0.0        12.2958 
 0.0        12.2194   0.0        12.3367 
 0.0        12.2461   0.0        12.3635 
 0.0        12.2669   0.0        12.3843 
 0.0        12.2842   0.0        12.4017 
 0.0        12.2993   0.0        12.4167 
 0.0        12.3125   0.0        12.43   
 0.0        11.589    0.0        11.589  

# Four Incumbents

## Investment Function with N = 4
The logic behind this investment function is same as the investment function in a market with two and three incumbents. Here we have to consider more cases than a market with two incumbents. Specifically, in a market with four incumbent firms, a firm might face an exit decision from one, two, three or none of its competitors. Moreover, each incumbent faces the threat of entry from at most one entrant in each period $x_e(2) = \infty$.

In [26]:
function fif(x)
    global s, χ0, x0, V0, p_e
    p = a*x0[address]./(1.0 .+ a*x0[address]) #ν1, ν2, ν3, ν4
    par_p_ν1 = a/(1.0 + a*x[1])^2
    EV = .0
    EV_e = .0
    if χ0[address[2:end]] == [1;1;1]
        for i=1:2,j=1:2, l=1:2, m=1:2, q=1:2
            τ[1,j] = ν[j] - ν[i]
            τ[2,l] = ν[l] - ν[i]
            τ[3,m] = ν[m] - ν[i]
            τ[4,q] = ν[q] - ν[i]
            pointer = [s1+τ[1,j]+ν[i] s2+τ[2,l]+ν[i]-1  s3+τ[3,m]+ν[i]-1 s4+τ[4,q]+ν[i]-1]*k_plus
            pointer = round(Int, pointer[1])
            prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
            ((1-ν[q])*(1-p[4]) + (ν[q]*p[4]))*
            ((1-ν[m])*(1-p[3]) + (ν[m]*p[3]))*
            ((1-ν[l])*(1-p[2]) + (ν[l]*p[2]))*
            ((1-ν[j])*(-par_p_ν1) + ν[j]*par_p_ν1)
            EV += prob*V0[pointer[1]]
        end
    # we have an entrant and at most one entrant = x_e(2) = ∞
    elseif χ0[address[2:end]] == [1;1;0]
        for i=1:2,j=1:2,l=1:2, m=1:2
            τ[1,j] = ν[j] - ν[i]
            τ[2,l] = ν[l] - ν[i]
            τ[3,m] = ν[m] - ν[i]
            pointer = [s_e-ν[i] s1+τ[1,j]+ν[i]-1 s2+τ[2,l]+ν[i]-1 s3+τ[3,m]+ν[i]-1]*k_plus
            pointer = round(Int, pointer[1])
            prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
            ((1-ν[m])*(1-p[3]) + (ν[m]*p[3]))*
            ((1-ν[l])*(1-p[2]) + (ν[l]*p[2]))*
            ((1-ν[j])*(1-p[1]) + ν[j]*p[1])
            EV_e += prob*V0[pointer[1]]
        end
        p_e = cdf(P_ce, EV_e[1])
        for i=1:2,j=1:2,l=1:2, m=1:2, q=1:2
            τ[1,j] = ν[j] - ν[i]
            τ[2,l] = ν[l] - ν[i]
            τ[3,q] = ν[q] - ν[i]
            pointer = [s1+τ[1,j]+ν[i] s2+τ[2,l]+ν[i]-1 s3+τ[3,q]+ν[i]-1 e[m]*(s_e-ν[i])+(1-e[m])*1-1]*k_plus
            pointer = round(Int, pointer[1])
            prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
            ((1-e[m])*(1-p_e) + e[m]*p_e)*
            ((1-ν[q])*(1-p[3]) + (ν[q]*p[3]))*
            ((1-ν[l])*(1-p[2]) + (ν[l]*p[2]))*
            ((1-ν[j])*(-par_p_ν1) + ν[j]*par_p_ν1)
            EV += prob*V0[pointer[1]]
        end
    elseif χ0[address[2:end]] == [1;0;0]
        for i=1:2,j=1:2,l=1:2
            τ[1,j] = ν[j] - ν[i]
            τ[2,l] = ν[l] - ν[i]
            pointer = [s_e-ν[i] s1+τ[1,j]+ν[i]-1 s2+τ[2,l]+ν[i]-1 0]*k_plus
            pointer = round(Int, pointer[1])
            prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
            ((1-ν[l])*(1-p[2]) + (ν[l]*p[2]))*
            ((1-ν[j])*(1-p[1]) + ν[j]*p[1])
            EV_e += prob*V0[pointer[1]]
        end
        p_e = cdf(P_ce, EV_e[1])
        for i=1:2,j=1:2,l=1:2, m=1:2
            τ[1,j] = ν[j] - ν[i]
            τ[2,l] = ν[l] - ν[i]
            pointer = [s1+τ[1,j]+ν[i] s2+τ[2,l]+ν[i]-1 e[m]*(s_e-ν[i])+(1-e[m])*1-1 0]*k_plus
            pointer = round(Int, pointer[1])
            prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
            ((1-e[m])*(1-p_e) + e[m]*p_e)*
            ((1-ν[l])*(1-p[2]) + (ν[l]*p[2]))*
            ((1-ν[j])*(-par_p_ν1) + ν[j]*par_p_ν1)
            EV += prob*V0[pointer[1]]
        end
    elseif χ0[address[2:end]] == [0;0;0]
        for i=1:2,j=1:2
            τ[1,j] = ν[j] - ν[i]
            pointer = [s_e-ν[i] s1+τ[1,j]+ν[i]-1 0 0]*k_plus
            pointer = round(Int, pointer[1])
            prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
            ((1-ν[j])*(1-p[1]) + ν[j]*p[1])
            EV_e += prob*V0[pointer[1]]
        end
        p_e = cdf(P_ce, EV_e[1])
        for i=1:2,j=1:2, m=1:2
            τ[1,j] = ν[j] - ν[i]
            pointer = [s1+τ[1,j]+ν[i] e[m]*(s_e-ν[i])+(1-e[m])*1-1 0 0]*k_plus
            pointer = round(Int, pointer[1])
            prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
            ((1-e[m])*(1-p_e) + e[m]*p_e)*
            ((1-ν[j])*(-par_p_ν1) + ν[j]*par_p_ν1)
            EV += prob*V0[pointer[1]]
        end
    end
    return EV[1] - c/β
end

fif (generic function with 1 method)

## Initial Values and Iteration with N = 4

In [27]:
k_plus = [1 ; k + 1 ; (k+1)^2 ; (k+1)^3]
s = [4;3;2;1]
s1 = s[1]
s2 = s[2]
s3 = s[3]
s4 = s[4]

ω = [s1  s2-1 s3-1 s4-1;
    s2 s1-1 s3-1 s4-1;
    s3 s1-1 s2-1 s4-1;
    s4 s1-1 s2-1 s3-1]
address = round.(Int, ω*k_plus)

# Initial Values, policy function
V0 = repeat( V_t , k+1, 1)
x0 = repeat( x_t , k+1, 1) 
χ0 = repeat( χ_t , k+1, 1) 
V1 = copy(V0)
V_check = copy(V0)
x1 = copy(x0)
χ1 = copy(χ0)
check = 0
iter = 1
initial_x = [0.1]
initial_p = [1.0; 1.0; 1.0; 1.0];

In [ ]:
iter = 1
check = 0
#fixed point loop
while iter < 201 && check == 0
    for ω1 =2:k, ω2 =1:k, ω3=1:ω2, ω4=1:ω3
        # set up
        s = [ω1; ω2; ω3; ω4]
        s1 = s[1]
        s2 = s[2]
        s3 = s[3]
        s4 = s[4]
        ω = [s1  s2-1 s3-1 s4-1;
            s2 s1-1 s3-1 s4-1;
            s3 s1-1 s2-1 s4-1;
            s4 s1-1 s2-1 s3-1]
        address = round.(Int, ω*k_plus)
        ω1_permutation = [s1 s2-1 s3-1 s4-1;
                        s1 s2-1 s4-1 s3-1;
                        s1 s3-1 s2-1 s4-1;
                        s1 s3-1 s4-1 s2-1;
                        s1 s4-1 s2-1 s3-1;
                        s1 s4-1 s3-1 s2-1;]
        permutation = round.(Int, ω1_permutation*k_plus)

        # investment, probability of increase in productivity, j-1 probability of increase of all firms
        solution = nlsolve( fif, initial_x , autodiff = :forward) #this function also returns p_e
        x_u = (solution.zero .>= 0).* solution.zero + (solution.zero .< 0)*0
        x_u = (solution.f_converged == 1)*x_u[1] + (solution.f_converged == 0)*0        
        p_ν1 = a*x_u[1]/(1.0 .+ a*x_u[1])
        pr = a*x0[address]./(1.0 .+ a*x0[address]) #ν1, ν2, ν3, ν4
        EV = .0

        # different regions of exit policy function
        if χ0[address[2:end]] == [1;1;1]
            initial_p = [1.0; 1.0; 1.0; 1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)
            for i=1:2,j=1:2, l=1:2, m=1:2, q=1:2
                τ[1,j] = ν[j] - ν[i]
                τ[2,l] = ν[l] - ν[i]
                τ[3,m] = ν[m] - ν[i]
                τ[4,q] = ν[q] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] s2+τ[2,l]+ν[i]-1  s3+τ[3,m]+ν[i]-1 s4+τ[4,q]+ν[i]-1]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-ν[q])*(1-pr[4]) + (ν[q]*pr[4]))*
                ((1-ν[m])*(1-pr[3]) + (ν[m]*pr[3]))*
                ((1-ν[l])*(1-pr[2]) + (ν[l]*pr[2]))*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        elseif χ0[address[2:end]] == [1;1;0]
            s = [s1; s2; s3]
            initial_p = [1.0; 1.0; 1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)
            for i=1:2,j=1:2,l=1:2, m=1:2, q=1:2
                τ[1,j] = ν[j] - ν[i]
                τ[2,l] = ν[l] - ν[i]
                τ[3,q] = ν[q] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] s2+τ[2,l]+ν[i]-1 s3+τ[3,m]+ν[i]-1 e[m]*(s_e-ν[i])+(1-e[m])*1-1]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-e[m])*(1-p_e) + e[m]*p_e)*
                ((1-ν[q])*(1-pr[3]) + (ν[q]*pr[3]))*
                ((1-ν[l])*(1-pr[2]) + ν[l]*pr[2])*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        elseif χ0[address[2:end]] == [1;0;0]
            s = [s1; s2]
            initial_p = [1.0; 1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)        
            for i=1:2,j=1:2,l=1:2, m=1:2
                τ[1,j] = ν[j] - ν[i]
                τ[2,l] = ν[l] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] s2+τ[2,l]+ν[i]-1 e[m]*(s_e-ν[i])+(1-e[m])*1-1 0]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-e[m])*(1-p_e) + e[m]*p_e)*
                ((1-ν[l])*(1-pr[2]) + (ν[l]*pr[2]))*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        elseif χ0[address[2:end]] == [0;0;0]
            s = [s1]
            initial_p = [1.0]
            solution = nlsolve( prices, initial_p, autodiff = :forward)
            p = solution.zero
            π_c = current_profit(p)        
            for i=1:2,j=1:2, m=1:2
                τ[1,j] = ν[j] - ν[i]
                pointer = [s1+τ[1,j]+ν[i] e[m]*(s_e-ν[i])+(1-e[m])*1-1 0 0]*k_plus
                pointer = round(Int, pointer[1])
                prob = ( (1-ν[i])*(1-δ) + ν[i]*δ )*
                ((1-e[m])*(1-p_e) + e[m]*p_e)*
                ((1-ν[j])*(1-p_ν1) + ν[j]*p_ν1)
                EV += prob*V0[pointer[1]]
            end
        end
        max_V, ind = findmax( [Φ ; π_c[1] - c*x_u[1] + β*EV[1] ])
        V1[permutation] .= max_V
        x1[permutation] .= (ind == 2)*x_u[1]
        χ1[permutation] .= (ind == 2)
        V_check[permutation] .= max_V
    end

    if norm( V1 - V0 )/(1 + norm(V1)) < 1e-3
        check = 1
        println( "iter: ", iter, " Percentage: ", norm( V1 - V0 )/(1 + norm(V1)) )
    end
    if iter%10 == 0
        println( "iter: ", iter, " Percentage: ", norm( V1 - V0 )/(1 + norm(V1)) )
        println( "iter: ", iter, " Norm: ", norm( V1 - V0 ) )
    end
    V0 = copy(V1)
    x0 = copy(x1)
    χ0 = copy(χ1)
    iter += 1
end

iter: 10 Percentage: 0.02189291674913873
iter: 10 Norm: 49.70474913746061
iter: 20 Percentage: 0.009025215213795273
iter: 20 Norm: 19.325013225018107

In [ ]:
[χ1[1:k+1] V1[1:k+1] χ1[10*(k+1)+1:11*(k+1)] V1[(k+1)+1:2*(k+1)]]